In [2]:
import numpy as np
import tensorflow as tf
import codecs
from os import listdir
import nltk
from gensim.models import Word2Vec
from random import randint

In [2]:
dirrs = ['sentiment/train/', 'sentiment/test/']
sent = []
for dirr in dirrs:
    print dirr
    l = listdir(dirr+'pos')
    print 'pos'
    for r in l:
        t = codecs.open(dirr+'pos/'+r,'r',encoding='utf8').read()
        sent.append(nltk.word_tokenize(t))
    l = listdir(dirr+'neg')
    print 'neg'
    for r in l:
        t = codecs.open(dirr+'neg/'+r,'r',encoding='utf8').read()
        sent.append(nltk.word_tokenize(t))

sentiment/train/
pos
neg
sentiment/test/
pos
neg


In [4]:
m = Word2Vec.load('embedding.model')
data = []
for s in sent:
    a = [m[x] for x in s]
    data.append(a)
data = np.array(data)
print data[0]

[array([ 0.42692333,  0.49234673, -0.2337437 ,  0.08326261, -0.17497903,
        0.09907874,  0.47981006,  0.18602276, -0.0423082 , -0.26891571,
       -0.06728674, -0.17129813, -0.57115322, -0.21877325, -0.02164491,
        0.55063874, -0.02465075, -0.25235951, -0.21444109,  0.02935429,
       -0.31684676,  0.3027955 ,  0.03426933,  0.22381417,  0.09271631,
        0.21298024, -0.36960912, -0.11948946, -0.39697048,  0.02169935,
        0.10538933,  0.57093334, -0.39342993,  0.05328443, -0.24864316,
        0.01399888,  0.27800217,  0.46416432,  0.47918996,  0.16683058,
        0.0450494 ,  0.0652392 ,  0.10963918, -0.03439279, -0.01305015,
        0.29122731,  0.30983371,  0.19998483,  0.28151354,  0.35815614,
        0.23857714, -0.22028457, -0.01336792, -0.00635747, -0.15266436,
        0.39810824, -0.15156427, -0.01333465, -0.09184736, -0.11323494,
       -0.08940745,  0.04438727,  0.59258318, -0.40914977,  0.17754713,
       -0.12229583,  0.60639495,  0.10949554,  0.48939106,  0.0

In [8]:
def sample(d):
    r = randint(0, data.shape[0] - 1)
    t = d[r]
    x = t[:-1]
    y = t[1:]
    return np.array(x), np.array(y)

s = sample(data)
s[0].shape, s[1].shape

((437, 200), (437, 200))

In [3]:
'''
debugging 
Wxh = tf.Variable([[1, 2], [3, 4], [5, 6], [7, 8]], dtype=tf.float32)
Whh = tf.Variable([[1, 2], [3, 4]], dtype=tf.float32)
Why = tf.Variable([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
'''
Wxh = tf.Variable(tf.random_normal([200, 100]))
Whh = tf.Variable(tf.random_normal([100, 100]))
Why = tf.Variable(tf.random_normal([100, 200]))

In [15]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    h = tf.squeeze(tf.matmul(i, Wxh)) + tf.squeeze(tf.matmul(p, Whh))
    return h

In [16]:
a = tf.placeholder(shape=[None, 200], dtype=tf.float32)
b = tf.placeholder(shape=[None, 200], dtype=tf.float32)
# initial = tf.Variable([1, 2], dtype=tf.float32)
initial = tf.Variable(tf.random_normal(100), dtype=tf.float32)
states = tf.scan(recurrence, a, initializer=initial)
outputs = tf.matmul(states, Why)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(outputs, b))
optimize_op = tf.train.AdagradOptimizer(0.01).minimize(loss)

AttributeError: 'NoneType' object has no attribute 'grad_context'

In [12]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    l, _ = sess.run([loss, optimize_op], {a: [[1, 2, 3, 4], [4, 5, 6, 7]], b: [[1,0,0],[0,0,1]]})
    print l

21.2652
